In [2]:
!clang -shared -o vector.so vector.c

vector.c:68:34: warning: sizeof on array function parameter will return size of 'VocabStore *' instead of 'VocabStore[]' [-Wsizeof-array-argument]
    int vocab_store_size = sizeof(vocab_store) / sizeof(vocab_store[0]);
                                 ^
vector.c:67:47: note: declared here
int c_fit_vectorize(char* texts[], VocabStore vocab_store[]) {
                                              ^
vector.c:68:48: warning: 'sizeof (vocab_store)' will return the size of the pointer, not the array itself [-Wsizeof-pointer-div]
    int vocab_store_size = sizeof(vocab_store) / sizeof(vocab_store[0]);
                           ~~~~~~~~~~~~~~~~~~~ ^
vector.c:67:47: note: pointer 'vocab_store' declared here
int c_fit_vectorize(char* texts[], VocabStore vocab_store[]) {
                                              ^
vector.c:69:27: warning: sizeof on array function parameter will return size of 'char **' instead of 'char *[]' [-Wsizeof-array-argument]
    int text_size = sizeof(texts) / size

In [3]:
import ctypes
import pandas as pd
clib = ctypes.CDLL('vector.so')

In [4]:
def carray(arr):
    return (ctypes.c_char_p * len(arr))()

df = pd.read_csv('./twitter_training.csv')
df = df.dropna()
text_data = df['text']


In [5]:

clib.c_transform.restype = ctypes.POINTER(ctypes.c_int)
clib.c_transform.argtypes = [ctypes.POINTER(ctypes.c_char_p), ctypes.c_int, ctypes.c_int]
transform = clib.c_transform
 
store = {}

In [6]:
from collections import defaultdict

class VocabStore(ctypes.Structure):
    _fields_ = [
        ("words", ctypes.c_char * 1000),
        ("count", ctypes.c_int)
    ]
store = defaultdict(int)
store[b"word1"] = 2
store[b"word2"] = 5
# Add more words and counts as needed

vocab_store = (VocabStore * len(store))()
for i, (word, count) in enumerate(store.items()):
    vocab_store[i].words = word
    vocab_store[i].count = count


In [7]:
texts = [b"text1", b"text2", b"text3"]  # Example texts
text_array = (ctypes.c_char_p * len(texts))()
text_array[:] = texts


In [ ]:
transformed_texts = clib.c_transform(text_array, len(vocab_store), len(texts))


: 

: 

In [19]:
transformed_texts.contents

c_int(643845136)

In [17]:
transformed_texts_list = [transformed_texts[i] for i in range(len(texts))]
transformed_texts_list

[643845136, 1, 644005360]

In [8]:

import ctypes
class BuildVectors(object):
    def __init__(self):
        self.clib = ctypes.CDLL('./vector.so')
        self.c_transform = self.clib.c_transform
        self.c_transform.argtypes = [ctypes.POINTER(ctypes.c_char_p),ctypes.c_int]
        #self.c_transform.restype=None
    def transform(self,texts):
        pass



NameError: name 'VocabStore' is not defined